<a href="https://colab.research.google.com/github/ssirin/SA_Retail_Market/blob/main/SA_supplier_model_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is written to analyze retail electricity market dynamics in Saudi Arabia. The simulation is based on these dynamics:

- There is projected demand and the single utility takes projected demand and estimates the prices.
- The price is taken by the retailer companies and used to estimate household demand. Households' electricity demand is a function of their demand income. In addition, somehouseholds have distributed energy technologies and storage. Some households are poor and they only have flat tariffs. The others have to choose a retailer and make a contract for a year. After the household demand is estimated the retailers submit to the generation company, and the utility will estimate price again. And this goes until the unserved demand becomes zero.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## This function works for generation
def estimate_generation_costs_hourly(residential_demand_24h, other_demand_24h, capacity, capacity_factors_24h, costs, voll):
    """
    Estimates hourly generation costs, marginal prices, and unserved demand for a 24-hour period.

    Parameters:
    - residential_demand_24h: List of residential demand values for each hour.
    - other_demand_24h: List of other demand values for each hour.
    - capacity: Tuple of capacities for solar, wind, gas, and oil.
    - capacity_factors_24h: List of tuples containing solar and wind capacity factors for each hour.
    - costs: Tuple of costs for solar, wind, gas (fixed and variable), and oil (fixed and variable).
    - voll: Value of lost load (VOLL).
    - storage_capacity: Capacity of the storage system in units.
    - storage_cost: Variable cost of using the storage system per unit.

    Returns:
    Tuple containing lists of hourly generation, average price, hourly marginal prices, and hourly unserved demand.
    """
    # Verify input lengths
    if not (len(residential_demand_24h) == len(other_demand_24h) == len(capacity_factors_24h) == 24):
        raise ValueError("Input lists must all contain 24 elements.")

    solar_capacity, wind_capacity, gas_capacity, oil_capacity, storage_capacity = capacity
    solar_cost, wind_cost, gas_fixed, gas_variable, oil_fixed, oil_variable,  storage_cost = costs

    # Initialize variables for results and storage state
    hourly_generation = []
    hourly_marginal_price = []
    hourly_unserved_demand = []
    total_cost = 0
    storage_level = 0

    # Process each hour
    for hour in range(24):
        solar_factor, wind_factor = capacity_factors_24h[hour]
        demand = residential_demand_24h[hour] + other_demand_24h[hour]

        # Calculate renewable generation
        solar_generation = solar_capacity * solar_factor
        wind_generation = wind_capacity * wind_factor
        renewable_generation = solar_generation + wind_generation

        if renewable_generation > demand:
            # Store excess generation
            excess_generation = renewable_generation - demand
            storage_space_available = storage_capacity - storage_level
            storage_level += min(excess_generation, storage_space_available)
            generation = demand
            marginal_cost = solar_cost if solar_generation >= wind_generation else wind_cost
            unserved_demand = 0
        else:
            generation = renewable_generation
            demand -= renewable_generation
            marginal_cost = solar_cost if solar_generation >= wind_generation else wind_cost

            # Use storage if needed and available
            if storage_level > 0 and demand > 0:
                from_storage = min(demand, storage_level)
                generation += from_storage
                storage_level -= from_storage
                demand -= from_storage
                marginal_cost = storage_cost

            # Use non-renewable sources if still needed
            additional_cost, additional_generation, demand, last_marginal_cost = calculate_non_renewable_generation(demand, gas_capacity, oil_capacity, gas_fixed, gas_variable, oil_fixed, oil_variable, voll)
            generation += additional_generation
            total_cost += additional_cost
            if additional_generation > 0:
                marginal_cost = last_marginal_cost
            unserved_demand = demand

        hourly_generation.append(generation)
        hourly_marginal_price.append(marginal_cost)
        hourly_unserved_demand.append(unserved_demand)

    # Calculate total and average costs
    total_generation = sum(hourly_generation)
    average_price = total_cost / total_generation if total_generation > 0 else voll

    return hourly_generation, average_price, hourly_marginal_price, hourly_unserved_demand

def calculate_non_renewable_generation(demand, gas_capacity, oil_capacity, gas_fixed, gas_variable, oil_fixed, oil_variable, voll):
    """
    Calculates the generation from non-renewable sources based on demand.

    Parameters:
    - demand: The remaining demand to be met after considering renewables and storage.
    - gas_capacity, oil_capacity: Capacities for gas and oil generation.
    - gas_fixed, gas_variable, oil_fixed, oil_variable: Fixed and variable costs for gas and oil.
    - voll: Value of Lost Load (VOLL).

    Returns:
    A tuple of total cost, additional generation, remaining demand, and the marginal cost.
    """
    total_cost = 0
    generation = 0
    marginal_cost = 0

    if demand > 0:
        # Gas generation
        if demand <= gas_capacity:
            generation = demand
            total_cost = gas_fixed + gas_variable * generation
            marginal_cost = gas_variable
            demand = 0
        else:
            generation += gas_capacity
            total_cost += gas_fixed + gas_variable * gas_capacity
            demand -= gas_capacity

            # Oil generation
            if demand <= oil_capacity:
                generation += demand
                total_cost += oil_fixed + oil_variable * demand
                marginal_cost = oil_variable
                demand = 0
            else:
                generation += oil_capacity
                total_cost += oil_fixed + oil_variable * oil_capacity
                demand -= oil_capacity
                marginal_cost = voll  # Any remaining demand is unserved

    return total_cost, generation, demand, marginal_cost




In [ ]:
# Example call to the function
hourly_generation, average_price, hourly_marginal_price, hourly_unserved_demand = estimate_generation_costs_hourly(
    demand_24h, capacity, capacity_factors_24h, costs, voll, storage_capacity, storage_cost)

print(f"Hourly Generation: {hourly_generation}")
print(f"Average Price: ${average_price:.2f} per MWh")
print(f"Hourly Marginal Price: {hourly_marginal_price}")
print(f"Hourly Unserved Demand: {hourly_unserved_demand}")
